In [1]:
import os
import gc
import re
import glob
import random
import sys
# sys.path.append('/kaggle/input/fast-soft-sort/')
import pickle
import numpy as np
# import cudf
# %load_ext cudf.pandas
import pandas as pd
import scipy
import scipy.stats as stats
# import cudf
# cudf.set_option("default_integer_bitwidth", 32)
# cudf.set_option("default_float_bitwidth", 32)
import matplotlib.pyplot as plt

import tifffile
from tqdm.auto import tqdm
tqdm.pandas()

from datetime import date, datetime
from pathlib import Path
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import KFold , GroupKFold, StratifiedGroupKFold
from sklearn import decomposition
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score

import imblearn
import itertools
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import lightgbm as lgb
import optuna.integration.lightgbm as op_lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from catboost import Pool
from gensim.models import word2vec, KeyedVectors
from scipy.spatial.distance import pdist, cdist
from scipy.spatial.distance import squareform
# from tabpfn import TabPFNClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
def apk(actual, predicted, k=10):
    if actual in predicted[:k]:
        return 1.0 / (np.where(predicted[:k]==actual)[0][0] + 1)
    return 0.0

def mapk(actual, predicted, k=10):
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

def mapk2(actual, predicted, k=10):
    return [apk(a, p, k) for a, p in zip(actual, predicted)]

In [3]:
#sessionが1つしかないものとsessionが複数あるものでわける
#sessionが1つで違う県の宿を予約することは少ない
#185386 #103312
#ans_yadの数を人気順とする？
#session数を人気順とする？

In [4]:
train_log = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/train_log.csv")
test_log = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/test_log.csv")
train_label = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/train_label.csv")
yado = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/yado.csv")

In [5]:
def yad_co_matrix(log, label):
    label = label.rename(columns={"yad_no": "ans_yad_no"})
    log = log.merge(label, on="session_id", how="left").drop(["seq_no"], axis=1)
    log["count"] = 1
    log["count"] = log.groupby(["session_id","yad_no"])["count"].cumsum()
    log = log[log["count"]==1]
    log = log[log["yad_no"]!=log["ans_yad_no"]]
    log2 = log.copy()
    log2["yad_no2"] = log2["yad_no"]
    log2["yad_no"] = log2["ans_yad_no"]
    log2["ans_yad_no"] = log2["yad_no2"]
    log2 = log2.drop(["yad_no2"], axis=1)
    log = pd.concat([log, log2]).drop(["session_id"],axis=1).reset_index(drop=True)
    log = log.groupby(["yad_no", "ans_yad_no"]).agg("count").reset_index()
    log = log.pivot_table(index="yad_no", columns="ans_yad_no")
    return log

In [6]:
def yad_co_matrix2(log, te_log, label):
    #train_log session,ans
    log1 = log[["session_id", "yad_no"]].copy().rename(columns={"yad_no":"ans_yad_no"})
    label = label.rename(columns={"yad_no": "ans_yad_no"})
    log2 = log.merge(label, on="session_id", how="left").drop(["seq_no"], axis=1)
    log2["count"] = 1
    log2["count"] = log2.groupby(["session_id","yad_no"])["count"].cumsum()
    log2 = log2[log2["count"]==1]
    log1 = log.merge(log1, on="session_id", how="left")
    log = pd.concat([log1, log2]).drop(["seq_no"], axis=1)
    log = log[log["yad_no"]!=log["ans_yad_no"]]
    log["count"] = 1
    log["count"] = log.groupby(["session_id", "yad_no", "ans_yad_no"])["count"].transform("count")
    log = log[log["count"]==1].drop(["session_id"],axis=1).reset_index(drop=True)
    #train_log yad_no ans_yad_noを入れ替え
    log1 = log.copy()
    log1["yad_no2"] = log1["yad_no"]
    log1["yad_no"] = log1["ans_yad_no"]
    log1["ans_yad_no"] = log1["yad_no2"]
    log1 = log1.drop(["yad_no2"], axis=1)
    log = pd.concat([log, log1]).reset_index(drop=True)
    
    #test_log session
    log1 = te_log[["session_id", "yad_no"]].copy().rename(columns={"yad_no":"ans_yad_no"})
    te_log = te_log.merge(log1, on="session_id", how="left")
    te_log = te_log[te_log["yad_no"]!=te_log["ans_yad_no"]]
    te_log["count"] = 1
    te_log["count"] = te_log.groupby(["session_id", "yad_no", "ans_yad_no"])["count"].transform("count")
    te_log = te_log[te_log["count"]==1].drop(["session_id"],axis=1).reset_index(drop=True)
    #test_log yad_no ans_yad_noを入れ替え
    log1 = te_log.copy()
    log1["yad_no2"] = log1["yad_no"]
    log1["yad_no"] = log1["ans_yad_no"]
    log1["ans_yad_no"] = log1["yad_no2"]
    log1 = log1.drop(["yad_no2"], axis=1)
    te_log = pd.concat([te_log, log1]).reset_index(drop=True)
    
    #test train結合
    log = pd.concat([log, te_log]).reset_index(drop=True).drop(["seq_no"], axis=1)
    log = log.groupby(["yad_no", "ans_yad_no"]).agg("count").reset_index()
    log = log.pivot_table(index="yad_no", columns="ans_yad_no")
    
    del te_log, log1, log2,
    return log
# #     te_log["count"] = 1
# #     te_log["count"] = te_log.groupby(["session_id","yad_no"])["count"].cumsum()
# #     te_log = te_log[te_log["count"]==1]
    
#     log = pd.concat([log, log2]).drop(["session_id"],axis=1).reset_index(drop=True)
    
#     log = log.groupby(["yad_no", "ans_yad_no"]).agg("count").reset_index()
#     log = log.pivot_table(index="yad_no", columns="ans_yad_no")
#     return log

In [7]:
def test_yad_co_matrix(te_log):
    #test_log session
    log1 = te_log[["session_id", "yad_no"]].copy().rename(columns={"yad_no":"ans_yad_no"})
    te_log = te_log.merge(log1, on="session_id", how="left")
    te_log = te_log[te_log["yad_no"]!=te_log["ans_yad_no"]]
    te_log["count"] = 1
    te_log["count"] = te_log.groupby(["session_id", "yad_no", "ans_yad_no"])["count"].transform("count")
    te_log = te_log[te_log["count"]==1].drop(["session_id"],axis=1).reset_index(drop=True)
    #test_log yad_no ans_yad_noを入れ替え
    log1 = te_log.copy()
    log1["yad_no2"] = log1["yad_no"]
    log1["yad_no"] = log1["ans_yad_no"]
    log1["ans_yad_no"] = log1["yad_no2"]
    log1 = log1.drop(["yad_no2"], axis=1)
    te_log = pd.concat([te_log, log1]).reset_index(drop=True).drop(["seq_no"], axis=1)
    
    add_df = pd.DataFrame()
    yads = te_log["yad_no"].unique()
    add_yad = []
    for i in range(1,13807):
        if i not in yads:
            add_yad.append(i)
    add_df["yad_no"] = np.array(add_yad)
    add_df["ans_yad_no"] = np.array(add_yad)
    add_df["count"] = -1
    
    te_log = te_log.groupby(["yad_no", "ans_yad_no"]).agg("count").reset_index()
    te_log = pd.concat([te_log, add_df]).reset_index(drop=True)
    te_pivot = te_log.pivot_table(index="yad_no", columns="ans_yad_no")
    
    return te_pivot

In [8]:
def label_merge(log, label):
    label = label.rename(columns={"yad_no":"ans_yad_no"})
    log = log.merge(label, how="left", on=["session_id"])
        
    return log

In [9]:
def yad_cd_candidate(log, label):
    label = label.rename(columns={"yad_no":"ans_yad_no"})
    log["count"] = 1
    log["count"] = log.groupby(["session_id", "yad_no"])["count"].cumsum()
    log = log[log["count"]==1]
    log = log.merge(label, how="left", on=["session_id"])
    log = log[log["yad_no"]!=log["ans_yad_no"]]
    
    yad_cd_dict={}
#     log = log[log["session_id"].isin(x)]
    print(len(log))
    for ya in tqdm(log["yad_no"].unique()):
        candidates = log[(log["yad_no"]==ya)]["ans_yad_no"].value_counts()[:10].index.tolist()
        yad_cd_dict[ya] = candidates
        
    return yad_cd_dict

In [10]:
def yad_score_culc(log, label, yado):
    label["label"] = 1
    log = log.merge(label, how="left", on=["session_id", "yad_no"])
    log["label"] = log["label"].fillna(0)
    log = log.merge(yado, how="left", on="yad_no")
    yado_score = log.query("label==1")["yad_no"].value_counts().reset_index()
    yado_score = yado_score.merge(yado, how="left", on='yad_no')
    log = log.drop(["label"],axis=1)
    return yado_score[["yad_no", "count", "lrg_cd", "sml_cd"]]

In [11]:
def yad_score_culc2(label, yado):
    label = label.groupby("yad_no")["session_id"].agg(["count"]).reset_index()
    label = label.merge(yado, on="yad_no", how="left")
    label = label.sort_values("count", ascending=False).reset_index(drop=True)
    return label[["yad_no", "count", "ken_cd", "lrg_cd", "sml_cd"]]

In [12]:
def yad_score_culc3(log, yado):
    log["count"] = 1
    log["count"] = log.groupby(["session_id", "yad_no"])["count"].transform("cumsum")
    log = log[log["count"]==1]
    log = log.groupby("yad_no")["session_id"].agg(["count"]).reset_index()
    log = log.merge(yado, on="yad_no", how="left")
    log = log.sort_values("count", ascending=False).reset_index(drop=True)
    return log[["yad_no", "count", "ken_cd", "lrg_cd", "sml_cd"]]

In [13]:
def yad_score_culc4(train_log, test_log, train_label, yado):
    train_log["count"] = 1
    train_log["count"] = train_log.groupby(["session_id", "yad_no"])["count"].transform("cumsum")
    train_log = train_log[train_log["count"]==1]
    
    test_log["count"] = 1
    test_log["count"] = test_log.groupby(["session_id", "yad_no"])["count"].transform("cumsum")
    test_log = test_log[test_log["count"]==1]  
    
    train_label["count"] = 1
    
    log = pd.concat([train_log, test_log, train_label]).reset_index(drop=True)
    log = log.groupby("yad_no")["session_id"].agg(["count"]).reset_index()
    log = log.merge(yado, on="yad_no", how="left")
    log = log.sort_values("count", ascending=False).reset_index(drop=True)
    return log[["yad_no", "count", "ken_cd", "lrg_cd", "sml_cd"]]

In [14]:
def seen_unseen_create(log):
    x = log.groupby("session_id")["seq_no"].count().reset_index()
    y = x[x["seq_no"]>1]["session_id"].unique()
    x = x[x["seq_no"]==1]["session_id"].unique()
    unseen_log = log[log["session_id"].isin(x)]
    seen_log = log[log["session_id"].isin(y)]
#     seq2_log = log[log["session_id"].isin(y2)]
#     seq3_log = log[log["session_id"].isin(y3)]
    
    return seen_log, unseen_log

In [15]:
train_seen_log, train_unseen_log = seen_unseen_create(train_log)

###共起行列をつくる###

#labelとsession合わせてどの宿同士が同じセッションにいるかをカウント
matrix = yad_co_matrix(train_log, train_label)
#どの宿同士が同じセッションにいるかをカウントtest_log only
test_matrix = test_yad_co_matrix(test_log)

###宿の人気順をつくる###

#train_labelだけ
train_label_yado_score = yad_score_culc2(train_label, yado)

#test_logだけ
test_log_yado_score = yad_score_culc3(test_log, yado)

#train_label,train_log,test_logすべて
all_yado_score = yad_score_culc4(train_log, test_log, train_label, yado)

#各セッションのラストを
# last_yad_df = train_log.groupby("session_id")["yad_no"].last().reset_index().rename(columns={"yad_no":"last_yad"})

In [16]:
def matrix_to_dic(matrix):
    matrix = matrix.fillna(-1)
    matrix_dic = {}
    for i in tqdm(range(len(matrix))):
        array = matrix.iloc[i].values
        ans = []
        inds = array.argsort()[::-1][:20]
        for ind in inds:
            if array[ind]>0:
                ans.append(ind+1)
        matrix_dic[i+1] = ans
    
    return matrix_dic

In [17]:
def test_matrix_to_dic(matrix, test_matrix):
    matrix = matrix.fillna(-1)
    test_matrix = test_matrix.fillna(-1)
    matrix_dic = {}
    for i in tqdm(range(len(test_matrix))):
        array = test_matrix.iloc[i].values
        _array = matrix.iloc[i].values
        ans = []
        inds = array.argsort()[::-1][:20]
        _inds = _array.argsort()[::-1][:20]
        
        ansamble=[inds, _inds]
        res = {}
        w = [1,0.01]
        for m in range(len(ansamble)):
            for n, v in enumerate(ansamble[m]):
                if v in res:
                    res[v] += (w[m]/(n+1))
                else:
                    res[v] = (w[m]/(n+1))
        res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())       
        
        for ind in res:
            if array[ind]>0:
                ans.append(ind+1)
        matrix_dic[i+1] = ans
    
    return matrix_dic

In [18]:
matrix_dic = matrix_to_dic(matrix)
test_matrix_dic = test_matrix_to_dic(matrix,test_matrix)

  0%|          | 0/13806 [00:00<?, ?it/s]

  0%|          | 0/13806 [00:00<?, ?it/s]

In [19]:
def extract_valid_yados(record):
    """test_logのセッションに含まれているyad_noを抽出"""
    return list(filter(lambda x: isinstance(x, int) and x >= 0, record))

def seen_log_create(log, last_yad_df, yado):
    log = log.merge(last_yad_df, on="session_id", how="left")
    log = log[log["yad_no"]!=log["last_yad"]]
    #log = log.sort_values(["session_id", "seq_no"], ascending=False)
    #log["seq_no"] = 1
    #log["seq_no"] = log.groupby("session_id")["seq_no"].cumsum()-1
    log = log.merge(yado, how="left", on="yad_no")
    log = log.sort_values(["session_id"], ascending=True).reset_index(drop=True)
#     log = log.merge(last_yad_df, on="session_id", how="left")
    return log

def pivot_create(log):
    pivot = log[["session_id", "seq_no", "yad_no"]].pivot_table(index='session_id', columns='seq_no')
    pivot = pivot.fillna(-1).astype(int)
    return pivot

def cd_dict_create(log, col):
    mode_df = log.groupby('session_id')[col].agg(lambda x: x.mode().iloc[0]).reset_index()
    m_dict = dict(zip(mode_df['session_id'], mode_df[col]))
    return m_dict

In [20]:
def amsamble(matrix_dic, yads):
    ab = []
    for y in yads:
        ab.append(matrix_dic[y])
    res = {}
    for m in range(len(ab)):
        for n, v in enumerate(ab[m]):
            if v in res:
                res[v] += (1/(n+1))
            else:
                res[v] = (1/(n+1))
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    return res

In [21]:
def candidate_add(candidates, tmp_yados, yad):
    for cand in candidates:
        if len(tmp_yados) >= 10:
            break
        if (cand not in tmp_yados)&(cand != yad):
            tmp_yados.append(cand)
    return tmp_yados

In [22]:
def seen_pred_create(pivot, log, matrix_dic, test_matrix_dic, sml_cd_dict, lrg_cd_dict, ken_cd_dict, test_yado_score, all_yado_score):
    preds = []
    for record in tqdm(pivot.itertuples()):
        session_id = record[0]
        session_yados = extract_valid_yados(record)
        session_yado_set = set(session_yados)
        
        tmp_yados = session_yados.copy()
        yads = log.query(f"session_id=='{session_id}'")["yad_no"].values
        last_yads = log.query(f"session_id=='{session_id}'")["last_yad"].values[0]
        yads = np.append(yads, last_yads)
        
        #matrixから候補出す
        candidates = amsamble(test_matrix_dic, yads)
        tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
        candidates = amsamble(matrix_dic, yads)
        tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
        #現時点での候補からmatrix候補出す
        if len(tmp_yados)<10:
            for t_yados in tmp_yados:
                candidates = test_matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
                if len(tmp_yados)>=10:
                    break
                candidates = matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
                if len(tmp_yados)>=10:
                    break
                    
        if len(tmp_yados)<10:
            #sml_cdの人気順(test_log only)
            target_sml_cd = sml_cd_dict.get(session_id)
            candidates = test_log_yado_score.query(f"sml_cd=='{target_sml_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
            #lrg_cdの人気順(test_log only)
            target_lrg_cd = lrg_cd_dict.get(session_id)
            candidates = test_log_yado_score.query(f"lrg_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
            #ken_cdの人気順(test_log only)
            target_ken_cd = ken_cd_dict.get(session_id)
            candidates = test_log_yado_score.query(f"ken_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
            #sml_cdの人気順(all)
            candidates = all_yado_score.query(f"sml_cd=='{target_sml_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
            #lrg_cdの人気順(all)
            candidates = all_yado_score.query(f"lrg_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1])
        
            #ken_cdの人気順(all)
            candidates = all_yado_score.query(f"ken_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, yads[-1]) 
        
        #あまったらtrainの人気
        extra_num = 10 - len(tmp_yados)
        for _ in range(extra_num):
            tmp_yados.append(3338) # trainで最も人気の宿
        
        pred = {
            f"predict_{idx}": cand
            for idx, cand in enumerate(tmp_yados)
        }
        preds.append(pred)
    pred_seen_df = pd.DataFrame(preds)
    return pred_seen_df

In [23]:
# train_seen_log = seen_log_create(train_seen_log, last_yad_df, yado)
# train_seen_pivot = pivot_create(train_seen_log)
# sml_cd_dict = cd_dict_create(train_seen_log, "sml_cd")
# lrg_cd_dict = cd_dict_create(train_seen_log, "lrg_cd")

In [24]:
# pred_seen_df = seen_pred_create(train_seen_pivot, train_log, matrix_dic, sml_cd_dict, lrg_cd_dict, yado_score2)

In [25]:
# pred_col = [f"predict_{i}" for i in range(10)]
# pred_seen_df["session_id"] = train_seen_pivot.index
# pred_seen_df = pred_seen_df.merge(train_label[["session_id", "yad_no"]], on="session_id", how="left")
# actual = pred_seen_df["yad_no"]
# pred_seen_df = pred_seen_df.drop(["yad_no"], axis=1)
# predicted = pred_seen_df[pred_col].values
# pred_seen_df["score"] = np.array(mapk2(actual, predicted, k=10))

In [26]:
def extract_valid_yados(record):
    """test_logのセッションに含まれているyad_noを抽出"""
    return list(filter(lambda x: isinstance(x, int) and x >= 0, record))

In [27]:
def unseen_cand_create(train_unseen_log, yado, test_log_yado_score, all_yado_score, matrix_dic, test_matrix_dic, num):
    preds = []
    kouho_df = pd.DataFrame()
    for ya in tqdm(train_unseen_log["yad_no"].unique()):
        _df = pd.DataFrame()
        tmp_yados = []
        
        #matrixから候補出す
        candidates = test_matrix_dic[ya]
        tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
        candidates = matrix_dic[ya]
        tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
        #現時点での候補からmatrix候補出す
        if len(tmp_yados)<10:
            for t_yados in tmp_yados:
                candidates = test_matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
                    
                candidates = matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
                    
        if len(tmp_yados)<10:
            for t_yados in tmp_yados:
                candidates = test_matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
                
                candidates = matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
                    
        if len(tmp_yados)<10:
            for t_yados in tmp_yados:
                candidates = test_matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
                
                candidates = matrix_dic[t_yados]
                tmp_yados=candidate_add(candidates, tmp_yados, ya)
                if len(tmp_yados)>=10:
                    break
        
        if len(tmp_yados)<10:
            #sml_cdの人気順(test_log only)
            target_sml_cd = yado[yado["yad_no"]==ya]["sml_cd"].values[0]
            candidates = test_log_yado_score.query(f"sml_cd=='{target_sml_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
            #lrg_cdの人気順(test_log only)        
            target_lrg_cd = yado[yado["yad_no"]==ya]["lrg_cd"].values[0]
            candidates = test_log_yado_score.query(f"lrg_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
            #ken_cdの人気順(test_log only)  
            target_ken_cd = yado[yado["yad_no"]==ya]["ken_cd"].values[0]
            candidates = test_log_yado_score.query(f"ken_cd=='{target_ken_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)

            #sml_cdの人気順(all)
            candidates = all_yado_score.query(f"sml_cd=='{target_sml_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
            #lrg_cdの人気順(all)        
            candidates = all_yado_score.query(f"lrg_cd=='{target_lrg_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
            #ken_cdの人気順(all)  
            candidates = all_yado_score.query(f"ken_cd=='{target_ken_cd}'")["yad_no"].tolist()
            tmp_yados=candidate_add(candidates, tmp_yados, ya)
        
        
        extra_num = num - len(tmp_yados)
        for _ in range(extra_num):
            tmp_yados.append(3338)
            
        _df["yad_no"] = np.array([ya]*num)
        _df["kouho_yad_no"] = np.array(tmp_yados)
        kouho_df = pd.concat([kouho_df, _df])
    kouho_df = kouho_df.reset_index(drop=True)
    return kouho_df

In [28]:
# train_unseen_log = train_unseen_log.merge(yado, on="yad_no", how="left")
# train_unseen_pivot = pivot_create(train_unseen_log)
# sml_cd_dict = cd_dict_create(train_unseen_log, "sml_cd")
# lrg_cd_dict = cd_dict_create(train_unseen_log, "lrg_cd")
# kouho_df = unseen_cand_create(train_unseen_log, yado, yado_score, matrix, 10)

In [29]:
def unseen_pred_create(pivot, kouho_df):
    preds = []
    for record in tqdm(pivot.itertuples()):
        session_id = record[0]
        yad_no = record[1]
        tmp_yados = kouho_df[kouho_df["yad_no"]==yad_no]["kouho_yad_no"].values.tolist()
        pred = {
            f"predict_{idx}": cand
            for idx, cand in enumerate(tmp_yados)
        }
        preds.append(pred)
    pred_unseen_df = pd.DataFrame(preds)
    return pred_unseen_df

In [30]:
# train_unseen_log = train_unseen_log.merge(yado, on="yad_no", how="left")
# train_unseen_pivot = pivot_create(train_unseen_log)
# sml_cd_dict = cd_dict_create(train_unseen_log, "sml_cd")
# lrg_cd_dict = cd_dict_create(train_unseen_log, "lrg_cd")
# kouho_df = unseen_cand_create(train_unseen_log, yado, yado_score, matrix, 10)
# pred_unseen_df = unseen_pred_create(train_unseen_pivot, kouho_df)

In [31]:
# pred_seen_df["session_id"] = train_seen_pivot.index
# pred_unseen_df["session_id"] = train_unseen_pivot.index
# _pred_df = pd.concat([pred_seen_df, pred_unseen_df])
# train_label = train_label.merge(_pred_df, on="session_id", how="left")

In [32]:
# pred_col = [f"predict_{i}" for i in range(10)]
# actual = train_label["yad_no"].values
# predicted = train_label[pred_col].values
# mapk(actual, predicted, k=10)

In [33]:
# l = train_label[train_label["session_id"].isin(pred_unseen_df["session_id"].unique())]
# pred_col = [f"predict_{i}" for i in range(10)]
# actual = l["yad_no"].values
# predicted = l[pred_col].values
# mapk(actual, predicted, k=10)

# Test

In [34]:
test_log = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/test_log.csv")
last_yad_df = test_log.groupby("session_id")["yad_no"].last().reset_index().rename(columns={"yad_no":"last_yad"})

In [35]:
test_seen_log, test_unseen_log = seen_unseen_create(test_log)

In [36]:
test_seen_log = seen_log_create(test_seen_log, last_yad_df, yado)
test_seen_pivot = pivot_create(test_seen_log)
sml_cd_dict = cd_dict_create(test_seen_log, "sml_cd")
lrg_cd_dict = cd_dict_create(test_seen_log, "lrg_cd")
ken_cd_dict = cd_dict_create(test_seen_log, "ken_cd")

In [37]:
test_seen_log

,session_id,seq_no,yad_no,last_yad,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,00001149e9c73985425197104712478c,0,3560,1959,0,205.0,1.0,0,NaN,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,52ca3d2824fc3cc90bd4274423badeed,87d9490219b3778f73c41b8176cf30d0
1,0000f17ae2628237d78d3a38b009d3be,0,757,8922,0,174.0,1.0,0,1.0,NaN,NaN,1.0,f0112abf369fb03cdc5f5309300913da,bd054cc265d68a400ccb976ac69c6463,dca13b5f308a0ae88ab8875a9ab56919,3267093e6bcad4a46af9d3e46350b22f
2,000174a6f7a569b84c5575760d2e9664,1,12341,13610,0,237.0,1.0,1,NaN,NaN,NaN,1.0,d86102dd9c232bade9a97dccad40df48,acb04522bdbc0582bf16a6a97567cc27,57b6663bea1ed3527b11e80be82d5235,a5e1136fd4ebaa1dd055df0a44841538
3,0002f6aa27bcf984eeb3cf07297a96a9,0,10904,10119,0,56.0,1.0,1,NaN,NaN,NaN,NaN,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,d944a4adb81c65f0392a8a273606ba72,0af2cfba7130b2b6ab8c227d3d948156
4,0003f18c0c221438a9f90a5f6a4e9330,1,12986,12425,0,201.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,c86352f5b57e80fe545cfec1fd8505a1,9d6a46da05976cab8ac2b8583215c665,568887ea1e1d8c3cf3c60b5be585aa6d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66192,fff922e03a34e95734f19373a820a9e4,0,8827,1066,0,203.0,1.0,0,1.0,NaN,NaN,1.0,f0112abf369fb03cdc5f5309300913da,072c85e1653e10c9c7dd065ad007125a,c918c473df6f5a01b3f8eb068530b020,efbb33bc0a3ce2c98a9d4ea7e8ff278e
66193,fffa7fb9ab0d006e3cc51b3b37d11e0b,0,3720,10774,0,105.0,1.0,0,NaN,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e
66194,fffa9f78f4fea69da07d47dd810913a1,0,4834,3913,0,159.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,aabf8b3cf6414702adfda5532705759a,da32f7dee35dbf732876e93f9cff9829
66195,fffd40e7248796057c7f660c10abe336,0,11561,2911,0,195.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,52ca3d2824fc3cc90bd4274423badeed,87d9490219b3778f73c41b8176cf30d0


In [38]:
test_seen_pivot

yad_no                      
seq_no                                0      1  2  3  4  5  6
session_id                                                   
00001149e9c73985425197104712478c   3560     -1 -1 -1 -1 -1 -1
0000f17ae2628237d78d3a38b009d3be    757     -1 -1 -1 -1 -1 -1
000174a6f7a569b84c5575760d2e9664     -1  12341 -1 -1 -1 -1 -1
0002f6aa27bcf984eeb3cf07297a96a9  10904     -1 -1 -1 -1 -1 -1
0003f18c0c221438a9f90a5f6a4e9330     -1  12986 -1 -1 -1 -1 -1
...                                 ...    ... .. .. .. .. ..
fff922e03a34e95734f19373a820a9e4   8827     -1 -1 -1 -1 -1 -1
fffa7fb9ab0d006e3cc51b3b37d11e0b   3720     -1 -1 -1 -1 -1 -1
fffa9f78f4fea69da07d47dd810913a1   4834     -1 -1 -1 -1 -1 -1
fffd40e7248796057c7f660c10abe336  11561     -1 -1 -1 -1 -1 -1
fffee3199ef94b92283239cd5e3534fa   1997     -1 -1 -1 -1 -1 -1

[60760 rows x 7 columns]

In [39]:
pred_seen_df = seen_pred_create(test_seen_pivot, test_seen_log, matrix_dic, test_matrix_dic,
                                sml_cd_dict, lrg_cd_dict, ken_cd_dict,
                                test_log_yado_score, all_yado_score)

0it [00:00, ?it/s]

In [40]:
test_unseen_log = test_unseen_log.merge(yado, on="yad_no", how="left")
test_unseen_pivot = pivot_create(test_unseen_log)
sml_cd_dict = cd_dict_create(test_unseen_log, "sml_cd")
lrg_cd_dict = cd_dict_create(test_unseen_log, "lrg_cd")
ken_cd_dict = cd_dict_create(test_unseen_log, "ken_cd")

In [41]:
test_unseen_pivot

,yad_no
seq_no,0
session_id,
0000e02747d749a52b7736dfa751e258,11984
00017e2a527901c9c41b1acef525d016,4621
00018613341f842d639d576d56258faa,385
00027c33bbdb2ee4a99543fc84df6cb5,4452
000300aea0d549a5a88ed1eafeed0675,1807
...,...
fffdd903643f8dc4e1c8629e6a38e340,5331
ffff62c6bb49bc9c0fbcf08494a4869c,12062


In [42]:
def unseen_cand_create_test(test_unseen_log, yado, yado_score, yado_score2, yado_score3, matrix, num):
    preds = []
    kouho_df = pd.DataFrame()
    for ya in tqdm(test_unseen_log["yad_no"].unique()):
        _df = pd.DataFrame()
        tmp_yados = []
        candidates = matrix.iloc[ya-1].values
        ind = candidates.argsort()
        ind = ind[~np.isnan(ind)]
        for i in reversed(ind):
            if candidates[i]>0:
                tmp_yados.append(i+1)
            if len(tmp_yados) >= num:
                break
                
        target_sml_cd = yado[yado["yad_no"]==ya]["sml_cd"].values[0]
        candidates = yado_score2.query(f"sml_cd=='{target_sml_cd}'")["yad_no"].tolist()
        for cand in candidates:
            if len(tmp_yados) >= num:
                break
            if cand not in tmp_yados:
                tmp_yados.append(cand)
                
        target_lrg_cd = yado[yado["yad_no"]==ya]["lrg_cd"].values[0]
        candidates = yado_score2.query(f"lrg_cd=='{target_lrg_cd}'")["yad_no"].tolist()
        for cand in candidates:
            if len(tmp_yados) >= num:
                break
            if cand not in tmp_yados:
                tmp_yados.append(cand)

        extra_num = num - len(tmp_yados)
        for _ in range(extra_num):
            tmp_yados.append(3338)
            
        _df["yad_no"] = np.array([ya]*num)
        _df["kouho_yad_no"] = np.array(tmp_yados)
        kouho_df = pd.concat([kouho_df, _df])
    kouho_df = kouho_df.reset_index(drop=True)
    return kouho_df

In [43]:
kouho_df = unseen_cand_create(test_unseen_log, yado, test_log_yado_score,
                              all_yado_score, matrix_dic, test_matrix_dic,10)

  0%|          | 0/11117 [00:00<?, ?it/s]

In [44]:
kouho_df[kouho_df["yad_no"]==338]

,yad_no,kouho_yad_no
88550,338,3084
88551,338,11609
88552,338,2462
88553,338,10596
88554,338,12332
88555,338,9721
88556,338,1140
88557,338,8948
88558,338,434
88559,338,6768


In [45]:
kouho_df[kouho_df["yad_no"]==kouho_df["kouho_yad_no"]]

,yad_no,kouho_yad_no


In [46]:
pred_unseen_df = unseen_pred_create(test_unseen_pivot, kouho_df)

0it [00:00, ?it/s]

In [47]:
pred_seen_df["session_id"] = test_seen_pivot.index
pred_unseen_df["session_id"] = test_unseen_pivot.index

In [48]:
_pred_df = pd.concat([pred_seen_df, pred_unseen_df])

In [49]:
test = pd.read_csv("/kaggle/input/atmacup16-recruit-dataset/test_session.csv")
test = test.merge(_pred_df, on="session_id", how="left")

In [50]:
test

,session_id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,00001149e9c73985425197104712478c,3560,9534,5466,6563,6488,5785,4714,2680,7461,11561
1,0000e02747d749a52b7736dfa751e258,11923,4066,143,8108,7014,613,6129,2439,10095,11237
2,0000f17ae2628237d78d3a38b009d3be,757,7710,9190,9910,1774,13570,410,10485,6721,3400
3,000174a6f7a569b84c5575760d2e9664,12341,6991,13521,3359,1542,10861,10746,4180,9319,5657
4,00017e2a527901c9c41b1acef525d016,2862,9020,763,11480,1448,607,5650,10826,13235,12029
...,...,...,...,...,...,...,...,...,...,...,...
174695,fffee3199ef94b92283239cd5e3534fa,1997,1885,7888,5744,12942,8771,11123,7641,9743,7062
174696,ffff62c6bb49bc9c0fbcf08494a4869c,2232,3802,3644,4014,1227,12432,899,4962,5331,6874
174697,ffff9a7dcc892875c7a8b821fa436228,13241,11037,13797,8143,2087,7308,13719,12939,4398,3955
174698,ffffb1d30300fe17f661941fd085b04b,3100,3002,2373,13672,1687,5513,4976,2407,6065,12281


In [51]:
pred_col = [f"predict_{i}" for i in range(10)]
test[pred_col].to_csv("20231217_01.csv", index=False)